<a href="https://colab.research.google.com/github/professorholowczak/Data_Warehousing/blob/main/nyc_opendata_311_download.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
# Downloading NYC OpenData 311 Data using Google Colab
# Likely need to install the sodapy module
%pip install sodapy

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.8/62.8 kB 1.9 MB/s eta 0:00:00
  Attempting uninstall: requests
    Found existing installation: requests 2.27.1
    Uninstalling requests-2.27.1:
      Successfully uninstalled requests-2.27.1


In [3]:
import pandas as pd
from sodapy import Socrata

# The Host Name for the API endpoint. The https:// will be added by Sodapy
data_url='data.cityofnewyork.us'
# The data set at the API endpoint (311 data in this case)
data_set='erm2-nwe9'
# The app token created at data.cityofnewyork.us/profile/edit/developer_settings
app_token='xyz123xyz123xyz123xyz123'    # Put your App token in the quotes

# Create the client to point to the API endpoint
client = Socrata(data_url, app_token) 
# Set the timeout to 600 seconds. If we get timeout errors, increase this value
client.timeout = 600


In [4]:
# Start at record 0 of the results
start = 0
# Fetch 2000 rows at a time
chunk_size = 2000
# Empty out our result list
results =[]
# Set up a filter so we don't try to fetch all of the data
where_clause="complaint_type LIKE '%Bicycle%' AND date_extract_y(created_date)=2017"
# See how many complaint records there are
record_count_dict = client.get(data_set, where=where_clause, select="COUNT(*)")
# Save as an integer
record_count = int(record_count_dict[0]['COUNT'])

# Loop until we have fetched all of the records
while True:
     # Fetch the set of records starting at 'start'
     results.extend( client.get(data_set, where=where_clause, offset=start, limit=chunk_size))
     # Move up the starting record for the next chunk
     start = start + chunk_size
     # Give some feedback
     print("Downloaded ", chunk_size, " records starting with ", start, " of ", record_count)
     # If we have fetched all of the records, bail out
     if (start > record_count ):
        break


Downloaded  2000  records starting with  2000  of  1952


In [ ]:
len(results)

1952

In [ ]:
# Convert the results list into a pandas data frame
df = pd.DataFrame.from_records(results)
# Save the dataframe to a CSV file
df.to_csv("311_bicycle_complaints_2017.csv.gz", index=False, compression="gzip")

In [ ]:
# Check some data
!zcat 311_bicycle_complaints_2017.csv.gz | head

unique_key,created_date,closed_date,agency,agency_name,complaint_type,descriptor,location_type,incident_zip,incident_address,street_name,cross_street_1,cross_street_2,address_type,city,facility_type,status,resolution_description,resolution_action_updated_date,community_board,bbl,borough,x_coordinate_state_plane,y_coordinate_state_plane,open_data_channel_type,park_facility_name,park_borough,latitude,longitude,location,intersection_street_1,intersection_street_2
38060975,2017-12-29T18:12:00.000,2018-03-06T12:00:00.000,DSNY,Department of Sanitation,Derelict Bicycle,14B Derelict Bicycle,Street,11226,1501 NOSTRAND AVENUE,NOSTRAND AVENUE,ERASMUS STREET,SNYDER AVENUE,ADDRESS,BROOKLYN,DSNY Garage,Closed,The Department of Sanitation plowed the area.,2018-03-06T12:00:00.000,17 BROOKLYN,3048840049,BROOKLYN,998283,175926,PHONE,Unspecified,BROOKLYN,40.64954177375759,-73.94942893128558,"{'latitude': '40.64954177375759', 'longitude': '-73.94942893128558', 'human_address': '{""address"": """", ""city"